In [1]:
# imports 

import numpy as np
import pandas as pd
import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

import seaborn as sns

# import re
# import math
# import copy

# from collections import defaultdict
# import csv
# import itertools
# import datetime 
# from datetime import datetime
# import time
# import dateutil.parser
# import pickle
# import random

# import gc
# import zipfile
# import sys, getopt
# import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

# import dask.dataframe as dd

InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last"

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import dask.dataframe as dd
import functools
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Load from parquet files 

# CREDITOR_FILE_NAME = 'creditor.parquet'
# SHIPMENT_FILE_NAME = 'shipment.parquet'
# REALIZATION_FILE_NAME = 'realization.parquet'

# shipment_file_name = '../data/shipment.parquet'
# creditor_file_name = '../data/creditor.parquet'
# realized_file_name = '../data/realization.parquet'
# columns = None

# shipments = (
#     dd
#     .read_parquet(shipment_file_name, columns=columns, engine='pyarrow')
#     .sample(frac=2e-3, random_state=88)  # TODO remove this line to process the full dataset
#     #.pipe(self._apply_queries, query_local_dicts)
#     .compute()
# ) # ~ 1.5 min

# creditors = (
#     dd
#     .read_parquet(creditor_file_name, columns=columns, engine='pyarrow')
#     .sample(frac=2e-3, random_state=88)  # TODO remove this line to process the full dataset
#     #.pipe(self._apply_queries, query_local_dicts)
#     .compute()
# ) # 1.5 min

# realized = (
#     dd
#     .read_parquet(realized_file_name, columns=columns, engine='pyarrow')
#     .sample(frac=2e-3, random_state=88)  # TODO remove this line to process the full dataset
#     #.pipe(self._apply_queries, query_local_dicts)
#     .compute()
# ) # ~5 min

In [ ]:
# Save dataframes as .csv -- for development

# Define and save dataframe data types

# shipment_data_types = shipments.dtypes
# creditor_data_types = creditors.dtypes
# realized_data_types = realized.dtypes

# shipment_data_types.to_csv('data/shipment_data_types.csv')
# creditor_data_types.to_csv('data/creditor_data_types.csv')
# realized_data_types.to_csv('data/realized_data_types.csv')

# Save dataframe to .csv 

# shipments.to_csv('data/shipments.csv', index = True)
# creditors.to_csv('data/creditors.csv', index = True)
# realized.to_csv('data/realized.csv', index = True)

In [3]:
# Load data types from .csv 

shipment_data_types = pd.read_csv('data/shipment_data_types.csv', index_col=0).iloc[:,0]
creditor_data_types = pd.read_csv('data/creditor_data_types.csv', index_col=0).iloc[:,0]
realized_data_types = pd.read_csv('data/realized_data_types.csv', index_col=0).iloc[:,0]

# shipment_data_types.equals(shipment_data_types_test)
# creditor_data_types.equals(creditor_data_types)
# realized_data_types.equals(realized_data_types)

In [4]:
# load dataframes from .csv 

ship0 = pd.read_csv('data/shipments.csv', low_memory = False, index_col=0).astype(shipment_data_types)
cred0 = pd.read_csv('data/creditors.csv', low_memory = False, index_col=0).astype(creditor_data_types)
real0 = pd.read_csv('realized.csv', low_memory = False, index_col=0).astype(realized_data_types)

# EDA

## Creditor EDA

In [ ]:
# creditors0 = creditors.copy()

creditors = creditors0.copy()

In [ ]:
creditors = creditors[[ 'vendor_name', 'terms_date_basis', 'invoice_number', 'invoice_amount', 'invoice_tax_amount', 'invoice_date', 'amount_paid', 'gross_amount', 
                       'amount_remaining', 'multiple_payment_schedule', 'purchasing_site_flag', 'po_number', 'multiple_po', 'invoice_status', 'holds', 'other_holds', 
                       'invoice_approval_status', 'pending_approver', 'working_days_pending_approver', 'iterations_in_workflow', 'date_invoice_received', 'creation_date', 'gl_date_invoice', 'due_date', 'currency', 'payment_date', 'cleared_date', 
                       'invoice_type', 'invoice_payment_status', 'payment_priority', 'source', 'invoice_id', 'line_type', 'invoice_accounted',
                       'date_invoice_approved', 'date_invoice_holds_released', 'n_scheduled_payment_holds', 'snapshot_date']]


In [ ]:
for c in creditors.columns:
    if isinstance(creditors[c].iloc[1], str):
        print(c, ':', creditors[c].unique().size, creditors[c].unique()[:10])
        print()

In [ ]:
creditors.shape
creditors.head()

In [ ]:
creditors.vendor_name.iloc[0]

In [ ]:
plt.rcParams["figure.figsize"] = [20,7]

creditors.groupby('payment_date')['amount_paid'].sum().plot()

## Realization EDA

In [ ]:
# realized0 = realized.copy()

realized = realized0.copy()

In [ ]:
realized = realized[['first', 'vendor_name', 'vendor_number', 'invoice_number', 'invoice_date', 'item_category',  
       'line_type', 'account', 'process', 'product',  'elimination_code', 'verrekenpartij', 
       'booked_date', 'currency', 'booked_amount', 'rate',  'is_booked', 'invoice_amount', # ****
       'gl_batch_name', 'gl_batch_posted_date', 'transfer_date', 'unit', 'unit_price', 'report_unit', # ****
       'invoice_type', 'snapshot_date']]

In [ ]:
realized = realized[realized.snapshot_date == realized.snapshot_date.iloc[5000]]

# realized.dtypes

In [ ]:
realized.shape
realized.invoice_number.unique().size
realized.invoice_date.unique().size
realized.booked_date.unique().size
realized['invoice_amount'].describe().round(2)

realized.gl_batch_posted_date.unique().size
realized.transfer_date.unique().size
realized.report_unit.unique().size
realized.invoice_type.unique()

realized.head(10)


In [ ]:
for c in realized.columns:
    if isinstance(realized[c].iloc[1], str):
        print(c, ':', realized[c].unique().size, realized[c].unique()[:10])
        print()


In [ ]:
creditors.head(3)

In [ ]:
# --- Notes ---
# Interesting columns

# item_category : 146
# line_type : 10
# project_number : 1758
# object_code : 1
# center_code : 1597
# account : 60
# process : 90
# product : 63
# elimination_code : 3
# verrekenpartij : 22
# currency : 9
# rate : 609
# is_booked : 2
# expenditure_line_number : 3
# expenditure_type : 7
# gl_batch_name : 1255
# po_type : 2
# contract_number : 1
# article_number : 2579
# article_description : 2620
# unit : 12
# report_unit : 100
# invoice_type : 3

In [ ]:
# dat_aoi = run_data[run_data.article_number == aoi].copy()
# dat_aoi = pd.merge(pd.DataFrame(dat_aoi.groupby(['year', 'week'])['net_qty'].sum()).reset_index(), dat_aoi[['year', 'week']].drop_duplicates())

# dat_aoi.year = [str(x) for x in dat_aoi.year]
# dat_aoi.week = [str(x) for x in dat_aoi.week]
# dat_aoi['date'] = [dt.datetime.strptime(x[0] + '-' + x[1] + '-1', "%Y-%W-%w") for x in zip(dat_aoi.year, dat_aoi.week)]

# plt.rcParams["figure.figsize"] = [20,7]
# dat_aoi[['date', 'net_qty']].set_index('date').plot(linewidth = 4)

In [ ]:
plt.rcParams["figure.figsize"] = [20,7]

realized.groupby('booked_date')['booked_amount'].sum().plot()

In [ ]:
realized.groupby('invoice_date')['booked_amount'].sum().plot()

# Merging 

In [27]:
cred = cred0.copy()
ship = ship0.copy()
real = real0.copy()

In [ ]:
# SQL query join
# -------------
# creditor
#    LEFT JOIN real ON creditor.INVOICE_ID = real.INVOICE_ID 
#        LEFT JOIN ship ON real.PO_NR = ship.PO_PO_NUMBER AND real.PO_LINE_NR = ship.LINE_LINE_NUM 

In [7]:
# 'invoice_id' in cred.columns
# 'invoice_id' in real.columns

# 'po_number' in real.columns
# 'po_number' in ship.columns

# 'po_line_number' in real.columns
# 'line_number' in ship.columns

In [35]:
# (1) reduce dataframes to common invoices
# (2) drop NA's, b/c they interfere with converting invoice_id to string

cred = cred[cred.invoice_id.isin(real.invoice_id)].dropna(subset = ['invoice_id'])
real = real[real.invoice_id.isin(cred.invoice_id)].dropna(subset = ['invoice_id'])

cred['invoice_id'] = cred['invoice_id'].astype(int).astype(str)
real['invoice_id'] = real['invoice_id'].astype(int).astype(str)

In [38]:
cred.shape
real.shape

(4892, 66)

(6520, 51)

In [39]:
cred_real = pd.merge(cred, real, how = 'left', on = 'invoice_id')

In [41]:
all3 = pd.merge(cred_real, ship, how = 'left', left_on=['po_number', 'po_line_number'], right_on = ['po_number', 'line_number'])



KeyError: 'po_number'

# Appendix

In [ ]:
# test = test.astype(dt)

# test.equals(realized)

# realized.po_number.iloc[0]
# test.po_number.iloc[0]

# neq = realized.iloc[0] == test.iloc[0]

# realized.columns[neq]